[View in Colaboratory](https://colab.research.google.com/github/cheonbok94/DeepLearningLecture/blob/master/Training_Neural_Network.ipynb)

In [0]:
import torch 
import torch.nn as nn #
import torch.nn.functional as F #
import torchvision # 이미지 관련 처리, Pretrained Model 관련된 Package 입니다. 
import torchvision.datasets as vision_dsets
import torchvision.transforms as T # 이미지 처리 (Vison) 관련된 transformation이 정의 되어 있습니다.
import torch.optim as optim # pytorch 에서 정의한 수 많은 optimization function 들이 들어 있습니다.
from torch.autograd import Variable 
from torch.utils import data

# MNIST Feed-forward Neural Network 

## Data Loader 불러오기

In [0]:

def MNIST_DATA(root='./',train =True,transforms=None ,download =True,batch_size = 32,num_worker = 1):

	print ("[+] Get the MNIST DATA")
	"""
  	torchvision.dataset 에는 우리가 많이 사용하는 데이터들을 쉽게 사용할 수 있도록 되어 있습니다. 
  	Machine Learning 에서 Hello world 라고 불리는 Mnist 데이터를 사용해 보겠습니다. 
  
  
	"""
	mnist_train = vision_dsets.MNIST(root = root,  #root 는 데이터의 저장 위치 입니다. 
									train = True, #Train 은 이 데이터가 train 데이터인지 아닌지에 대한 정보입니다. 
									transform = T.ToTensor(), # 얻어낸 데이터를 pytorch가 계산 할 수 있는 Tensor 로 변환해 줍니다. 
									download = True)  # 데이터를 다운로드 할지 여부를 물어봅니다. 
	mnist_test = vision_dsets.MNIST(root = root,
									train = False,  # Test Data를 가져오기에 Train =False 를 줘야 합니다. 
									transform = T.ToTensor(),
									download = True)
	"""
  	Data Loader 는 데이터와 batch size의 정보를 바탕으로 매 iteration 마다 주어진 데이터를 원하는 batch size 만큼 반환해주는 iterator입니다. 
  	* Practical Guide : Batch size 는 어느정도가 좋나요? -- 클 수록 좋다는 소리가 있습니다. 하지만 gpu memeory 사이즈 한계에 의해 기본적으로 batch size 가 
  	커질 수록 학습에 사용되는 gpu memory 사이즈가 큽니다. (Activation map을 저장해야 하기 때문입니다.) 기본적으로 2의 배수로 저장하는 것이 좋습니다.(Bit size 관련) 
  
	"""
	trainDataLoader = data.DataLoader(dataset = mnist_train,  # DataSet은 어떤 Data를 제공해 줄지에 대한 정보입니다. 여기서는 Training DATA를 제공합니다. 
									batch_size = batch_size, # batch size 정보를 꼭 줘야 합니다. 한 Batch 당 몇 개의 Data 를 제공할지에 대한 정보입니다. 
									shuffle =True, # Training의 경우 Shuffling 을 해주는 것이 성능에 지대한 영향을 끼칩니다. 꼭 True 를 줘야 합니다. 
									num_workers = 1) # num worker의 경우 데이터를 로드하는데 worker를 얼마나 추가하겠는가에 대한 정보입니다. 

	testDataLoader = data.DataLoader(dataset = mnist_test, # Test Data Loader 이므로 Test Data를 인자로 전달해줍니다.
									batch_size = batch_size, # 마찬가지로 Batch size 를 넣어줍니다. 
									shuffle = False, # shuffling 이 굳이 필요하지 않으므로 false를 줍니다. 
									num_workers = 1) #
	print ("[+] Finished loading data & Preprocessing")
	return mnist_train,mnist_test,trainDataLoader,testDataLoader

In [107]:
trainDset,testDset,trainDataLoader,testDataLoader= MNIST_DATA(batch_size = 32)  # Data Loader 를 불러 옵니다. 

[+] Get the MNIST DATA
[+] Finished loading data & Preprocessing


## Train Function 

In [0]:
def train_network(net,optimizer,trainloader):
  for epoch in range(4):  # loop over the dataset multiple times

      running_loss = 0.0 # running loss를 저장하기 위한 변수입니다. 
      for i, data in enumerate(trainloader, 0): # 한 Epoch 만큼 돕니다. 매 iteration 마다 정해진 Batch size 만큼 데이터를 뱉습니다. 
          # get the inputs
          inputs, labels = data # DataLoader iterator의 반환 값은 input_data 와 labels의 튜플 형식입니다. 
          inputs = Variable(inputs).cuda() # Pytorch에서 nn.Module 에 넣어 Backprop을 계산 하기 위해서는 Variable로 감싸야 합니다.
          labels = Variable(labels).cuda()
          # zero the parameter gradients
          optimizer.zero_grad()    #  현재 기존의 backprop을 계산하기 위해서 저장했던 activation buffer 를 비웁니다. Q) 이걸 안 한다면?

          # forward + backward + optimize
          outputs = net(inputs) # input 을 넣은 위 network 로 부터 output 을 얻어냅니다. 
          loss = criterion(outputs, labels) # loss fucntion에 주어진 target과 output 의 score를 계산하여 반환합니다. 
          loss.backward() # * Scalar Loss value를 Backward() 해주게 되면 주어진 loss값을 바탕으로 backpropagation이 진행됩니다. 
          optimizer.step() # 계산된 Backprop 을 바탕으로 optimizer가 gradient descenting 을 수행합니다. 

          # print statistics
          running_loss += loss.data[0]
          if i % 500 == 499:    # print every 2000 mini-batches
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 500))
              running_loss = 0.0

  print('Finished Training')

## Test Function

In [0]:
def test(model,test_loader):
  model.eval() # Eval Mode 왜 해야 할까요?  --> nn.Dropout BatchNorm 등의 Regularization 들이 test 모드로 들어가게 되기 때문입니다. 
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = Variable(data).cuda(), Variable(target).cuda()  # 기존의 train function의 data 처리부분과 같습니다. 
    output = model(data) 
    pred = output.max(1, keepdim=True)[1] # get the index of the max 
    correct += pred.eq(target.view_as(pred)).sum().data[0] # 정답 데이터의 갯수를 반환합니다. 

  test_loss /= len(test_loader.dataset)
  print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.format(
      correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

## Neural Network  + Activation Function

### 간단한 Neural Network 를 만들어 봅시다. (1)
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Sigmoid 

Layer 2 - input: 30 output:10

Cross Entropy Loss  + SGD optimizer 

In [0]:

class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = F.sigmoid(self.fc0(x)) # 28*28 -> 30 -> Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

#### Optimizer 
Optimizer 의 경우 기본적으로 torch.optim 안에 존재합니다. 다양한 optimziers 가 정의되어 있습니다. 

기본적으로 다음과 같은 구성을 따릅니다. optim.{Optimzier 이름}({Network Parameters},lr ={learning rate })

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [128]:
train_network(mnist_net,optimizer,trainDataLoader) # 4 Epoch 정도 학습을 진행해봅니다. 

[1,   500] loss: 2.366
[1,  1000] loss: 2.313
[1,  1500] loss: 2.285
[2,   500] loss: 2.256
[2,  1000] loss: 2.243
[2,  1500] loss: 2.230
[3,   500] loss: 2.205
[3,  1000] loss: 2.194
[3,  1500] loss: 2.177
[4,   500] loss: 2.149
[4,  1000] loss: 2.133
[4,  1500] loss: 2.118
Finished Training


In [129]:
test(mnist_net,testDataLoader) # Test 정확도를 출력해 봅니다. 


Test set:  Accuracy: 6152/10000 (62%)



### 간단한 Neural Network 를 만들어 봅시다. (2)
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - tanh 

Layer 2 - input: 30 output:10

Cross Entropy Loss  + SGD optimizer 

In [0]:

class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = F.tanh(self.fc0(x)) # 28*28 -> 30 -> Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [118]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 2.222
[1,  1000] loss: 2.078
[1,  1500] loss: 1.934
[2,   500] loss: 1.700
[2,  1000] loss: 1.581
[2,  1500] loss: 1.464
[3,   500] loss: 1.301
[3,  1000] loss: 1.211
[3,  1500] loss: 1.146
[4,   500] loss: 1.028
[4,  1000] loss: 0.978
[4,  1500] loss: 0.942
Finished Training


In [119]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 8327/10000 (83%)



### 간단한 Neural Network 를 만들어 봅시다. (3)
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Relu

Layer 2 - input: 30 output:10

Cross Entropy Loss  + SGD optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        # If the size is a square you can only specify a single number
        x = x.view(-1,28*28)
        x = F.relu(self.fc0(x))
        x = self.fc1(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [124]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 2.269
[1,  1000] loss: 2.185
[1,  1500] loss: 2.071
[2,   500] loss: 1.838
[2,  1000] loss: 1.685
[2,  1500] loss: 1.538
[3,   500] loss: 1.297
[3,  1000] loss: 1.169
[3,  1500] loss: 1.064
[4,   500] loss: 0.920
[4,  1000] loss: 0.862
[4,  1500] loss: 0.808
Finished Training


In [125]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 8473/10000 (85%)



### Q) 성능차이가 존재하나요? 존재한다면 무슨 이유일까요?

### 간단한 Neural Network 를 만들어 봅시다. (4) 
특징 : 3개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 40 + Activation Fucntion - sigmoid 

Layer 2 - input: 40 output: 30

Layer 3 - input: 30 output : 10

Cross Entropy Loss  + SGD optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,40) # Layer 1
        self.fc1 = nn.Linear(40, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = F.sigmoid(self.fc0(x))
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [136]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 2.317
[1,  1000] loss: 2.307
[1,  1500] loss: 2.304
[2,   500] loss: 2.300
[2,  1000] loss: 2.301
[2,  1500] loss: 2.301
[3,   500] loss: 2.300
[3,  1000] loss: 2.300
[3,  1500] loss: 2.300
[4,   500] loss: 2.299
[4,  1000] loss: 2.299
[4,  1500] loss: 2.298
Finished Training


In [137]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 1135/10000 (11%)



### Q) 학습이 잘 되나요???? 안 된다면 왜 안될까요?

### 간단한 Neural Network 를 만들어 봅시다. (5) 
특징 : 3개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 40 + Activation Fucntion - Relu 

Layer 2 - input: 40 output: 30

Layer 3 - input: 30 output : 10

Cross Entropy Loss  + SGD optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,40) #Layer 1 
        self.fc1 = nn.Linear(40, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.fc0(x)) # Layer 1
        x = F.relu(self.fc1(x)) # Layer 2
        x = self.fc2(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [140]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 2.306
[1,  1000] loss: 2.295
[1,  1500] loss: 2.283
[2,   500] loss: 2.255
[2,  1000] loss: 2.233
[2,  1500] loss: 2.206
[3,   500] loss: 2.137
[3,  1000] loss: 2.085
[3,  1500] loss: 2.017
[4,   500] loss: 1.854
[4,  1000] loss: 1.736
[4,  1500] loss: 1.613
Finished Training


In [141]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 6856/10000 (69%)



#### (4)와 차이가 존재하나요? 그렇다면 왜 그럴까요? (3) 이랑은 비교해보면 어떻나요? 

### 간단한 Neural Network 를 만들어 봅시다. (6) 
특징 : 3개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 40 + Activation Fucntion - Relu 

Layer 2 - input: 40 output: 30

Layer 3 - input: 30 output : 10

Cross Entropy Loss  + **Adam** optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,40) #Layer 1 
        self.fc1 = nn.Linear(40, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.fc0(x)) # Layer 1
        x = F.relu(self.fc1(x)) # Layer 2
        x = self.fc2(x) # Layer 3 
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [144]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.702
[1,  1000] loss: 0.322
[1,  1500] loss: 0.276
[2,   500] loss: 0.217
[2,  1000] loss: 0.192
[2,  1500] loss: 0.191
[3,   500] loss: 0.160
[3,  1000] loss: 0.154
[3,  1500] loss: 0.145
[4,   500] loss: 0.123
[4,  1000] loss: 0.126
[4,  1500] loss: 0.123
Finished Training


In [146]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9604/10000 (96%)



### 간단한 Neural Network 를 만들어 봅시다. (7) Layer 를 줄여볼까요? 
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Relu 

Layer 2 - input: 30 output : 10

Cross Entropy Loss  + **Adam** optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30) #Layer 1 
        self.fc1 =  nn.Linear(30, 10) # Layer 2

    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.fc0(x)) # Layer 1
        x = self.fc1(x) # Layer 2
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [152]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.669
[1,  1000] loss: 0.329
[1,  1500] loss: 0.293
[2,   500] loss: 0.241
[2,  1000] loss: 0.223
[2,  1500] loss: 0.209
[3,   500] loss: 0.173
[3,  1000] loss: 0.182
[3,  1500] loss: 0.161
[4,   500] loss: 0.144
[4,  1000] loss: 0.142
[4,  1500] loss: 0.137
Finished Training


In [153]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9587/10000 (96%)



### 간단한 Neural Network 를 만들어 봅시다. (7) Batch Norm 을 줘 볼까요?
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Relu  + Batch Norm

Layer 2 - input: 30 output : 10

Cross Entropy Loss  + **Adam** optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30) #Layer 1
        self.bn0 = nn.BatchNorm1d(30) # BatchNorm 
        self.fc1 =  nn.Linear(30, 10) # Layer 2
    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.bn0(self.fc0(x))) # Layer 1
        x = self.fc1(x) # Layer 2
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [160]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.896
[1,  1000] loss: 0.366
[1,  1500] loss: 0.293
[2,   500] loss: 0.234
[2,  1000] loss: 0.217
[2,  1500] loss: 0.200
[3,   500] loss: 0.172
[3,  1000] loss: 0.183
[3,  1500] loss: 0.176
[4,   500] loss: 0.150
[4,  1000] loss: 0.157
[4,  1500] loss: 0.150
Finished Training


In [161]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9620/10000 (96%)



### 간단한 Neural Network 를 만들어 봅시다. (8) 더 깊은 레이어에 Batch Norm 을 줘 볼까요?
특징 : 2개의 Layer를 가지는 Neural Network

<구성>  

Layer 1 - input:28*28 , output : 40 + Activation Fucntion - Relu + BatchNorm

Layer 2 - input: 40 output: 30 + Activation Fucntion - Relu  + BatchNorm

Layer 3 - input: 30 output : 10

Cross Entropy Loss  + **Adam** optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,40) #Layer 1 
        self.bn0 = nn.BatchNorm1d(40) #BatchNorm1 
        self.fc1 = nn.Linear(40, 30) # Layer 2
        self.bn1 = nn.BatchNorm1d(30) #BatchNorm1 
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.bn0(self.fc0(x))) # Layer 1
        x = F.relu(self.bn1(self.fc1(x))) # Layer 2
        x = self.fc2(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [185]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.845
[1,  1000] loss: 0.282
[1,  1500] loss: 0.220
[2,   500] loss: 0.164
[2,  1000] loss: 0.159
[2,  1500] loss: 0.155
[3,   500] loss: 0.135
[3,  1000] loss: 0.128
[3,  1500] loss: 0.121
[4,   500] loss: 0.103
[4,  1000] loss: 0.115
[4,  1500] loss: 0.109
Finished Training


In [186]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9707/10000 (97%)



#### Batch Normalization 을 적용한 (7)과 (6)을 비교해보고 (8) 과 (5)를 비교해보면 어떻나요? 학습이 어떻게 달라졌을까요? 

### Let's Do it - 성능을 한번 끝까지 높여볼까요~? 마음대로 한번 최고 성능을 찍어봅시다

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        
        self.fc0 = nn.Linear(28*28,16*5*5)
        self.bn0 = nn.BatchNorm1d(16*5*5)
        self.fc1 = nn.Linear(16 * 5 * 5, 84)
        self.bn1 = nn.BatchNorm1d(84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
      
        # If the size is a square you can only specify a single number
        x = x.view(-1,28*28)
        x = F.relu(self.bn0(self.fc0(x)))
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.fc3(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001)

In [171]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.469
[1,  1000] loss: 0.182
[1,  1500] loss: 0.149
[2,   500] loss: 0.105
[2,  1000] loss: 0.100
[2,  1500] loss: 0.096
[3,   500] loss: 0.062
[3,  1000] loss: 0.070
[3,  1500] loss: 0.071
[4,   500] loss: 0.049
[4,  1000] loss: 0.056
[4,  1500] loss: 0.056
Finished Training


In [172]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9791/10000 (98%)



## Practical Guide Pytorch nn.Sequential 



```
x = F.relu(self.bn0(self.fc0(x)))
x = F.relu(self.bn1(self.fc1(x)))
```
너무 복잡하지 않나요?  그냥 x = self.fc(x) 쉽게 해버리면 안 될까요?

Solution : nn.Sequential + 자매품 nn.ModuList


In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        
        layer_list = [] # 이 리스트에 모든 Layer 를 순차적으로 append 해보겠습니다. 
        layer_list.append(nn.Linear(28*28,40)) #Layer 1 
        layer_list.append(nn.BatchNorm1d(40))#BatchNorm1 
        layer_list.append(nn.Linear(40, 30)) # Layer 2
        layer_list.append(nn.BatchNorm1d(30)) #BatchNorm1 
        layer_list.append(nn.Linear(30, 10)) # Layer 3
        self.net  = nn.Sequential(*layer_list) # nn.Sequential 에 layer list를 넘겨 줍니다.
    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = self.net(x) # 넣은 순서대로 적용이 됩니다. 
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [181]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.728
[1,  1000] loss: 0.398
[1,  1500] loss: 0.375
[2,   500] loss: 0.338
[2,  1000] loss: 0.344
[2,  1500] loss: 0.336
[3,   500] loss: 0.328
[3,  1000] loss: 0.317
[3,  1500] loss: 0.320
[4,   500] loss: 0.313
[4,  1000] loss: 0.308
[4,  1500] loss: 0.326
Finished Training


In [182]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9189/10000 (92%)



#### 연습해 봅시다 ! 

특징 : 2개의 Layer를 가지는 Neural Network <구성>

Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Relu + Batch Norm

Layer 2 - input: 30 output : 10

Cross Entropy Loss + Adam optimizer

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        
        layer_list = [] # 이 리스트에 모든 Layer 를 순차적으로 append 해보겠습니다. 
        layer_list.append(nn.Linear(28*28,30)) #Layer 1 
        layer_list.append(nn.BatchNorm1d(30)) #BatchNorm1 
        layer_list.append(nn.Linear(30, 10)) # Layer 2
        self.net  = nn.Sequential(*layer_list) # nn.Sequential 에 layer list를 넘겨 줍니다.
    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = self.net(x) # 넣은 순서대로 적용이 됩니다. 
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [189]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.760
[1,  1000] loss: 0.383
[1,  1500] loss: 0.352
[2,   500] loss: 0.330
[2,  1000] loss: 0.329
[2,  1500] loss: 0.324
[3,   500] loss: 0.305
[3,  1000] loss: 0.308
[3,  1500] loss: 0.323
[4,   500] loss: 0.301
[4,  1000] loss: 0.302
[4,  1500] loss: 0.313
Finished Training


In [190]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9204/10000 (92%)

